In [2]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
from utils.utils import *

In [8]:
means_R2 = np.zeros((len(subj_list), len(rois)))

for i, subj in enumerate(subj_list):
    lh_R2_path = os.path.join(label_dir, 'freesurfer', subj, 'label', 'lh.prfR2.mgz')
    rh_R2_path = os.path.join(label_dir, 'freesurfer', subj, 'label', 'rh.prfR2.mgz')

    lh_R2 = nib.load(lh_R2_path).get_fdata().squeeze()
    rh_R2 = nib.load(rh_R2_path).get_fdata().squeeze()

    maskdata_lh_path = os.path.join(mask_dir, subj, f'lh.{subj}.testrois.mgz')
    maskdata_rh_path = os.path.join(mask_dir, subj, f'rh.{subj}.testrois.mgz')

    maskdata_lh = nib.load(maskdata_lh_path).get_fdata().squeeze()
    maskdata_rh = nib.load(maskdata_rh_path).get_fdata().squeeze()

    maskdata_lh[maskdata_lh > 15] = 0
    maskdata_rh[maskdata_rh > 15] = 0

    lh_R2 = lh_R2[maskdata_lh != 0]
    rh_R2 = rh_R2[maskdata_rh != 0]

    mask_file = os.path.join(mask_dir, subj, f'short.reduced.{subj}.testrois.npy')
    mask = np.load(mask_file, allow_pickle=True)
    R2 = np.hstack((lh_R2, rh_R2))
    assert mask.shape == R2.shape

    means_R2[i, :] = pd.Series(R2).groupby(mask).mean().to_numpy()

pd.DataFrame(means_R2, index=subj_list, columns=rois.keys())

    
    

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
subj01,46.182376,47.083756,51.636433,37.363702,27.717237,25.786176,9.377214,2.125349,34.071641,13.448743,19.589791,6.995954
subj02,47.632050,37.196521,39.561145,33.733166,18.451878,13.913630,5.851344,0.147087,29.568625,15.178284,11.069810,9.650835
subj03,47.692742,45.502800,45.436416,35.774878,26.623350,9.171463,-3.440064,-1.788429,30.322293,7.580125,-2.050293,-18.186402
subj04,42.224371,42.701160,35.851453,31.382560,15.024665,8.681799,8.431121,5.525737,31.482744,11.053558,11.450596,8.515602
subj05,59.961843,52.722076,51.170574,41.122391,29.194832,21.446724,13.900348,4.810795,37.154131,13.148699,-2.905746,20.001192
subj06,51.706530,51.471710,43.971635,44.790459,25.086231,11.525078,2.704639,-1.560302,35.278147,4.268366,9.317652,1.380007
subj07,41.885287,44.798774,46.020299,30.818452,25.692193,11.264747,1.411465,-1.324455,37.151549,10.276144,10.933102,5.761503
subj08,44.812764,46.366707,45.833612,46.811948,27.895577,12.388309,4.524923,2.297426,36.411649,19.607232,20.027980,16.474295
